## 💼 "All-Weather" Risk-Parity Portfolio Framework

### 📥 Setups and ETFs Selection

Installation and Import of required packages

In [457]:
# !pip install -r requirements.txt

In [458]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go

Initialization of portfolio parameters

In [459]:
TICKERS: list[str] = ['SPY', 'TLT', 'GLD', 'TIP', 'BIL']  # Equities, Fixed Income, Commodities, Inflation-protected, Cash
START_DATE: str = '2015-01-01'
END_DATE: str = '2025-06-27'
VOL_LOOKBACK: int = 126  # Volatility Lookback Period Length

### 🏗️ Data Download

Function for fetching daily price data

In [460]:
def get_etf_data(tickers: list[str], start: str, end: str):
    """
    Fetches financial data from Yahoo Finance API for given tickers and date range.
    
    Parameters:
        tickers (list[str]): U.S. equity ticker symbols
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format

    Returns:
        pd.DataFrame: DataFrame with date-indexed adjusted closing price data
    """
    
    data = yf.download(tickers, start, end, auto_adjust=True)['Close']
    return data.dropna()

ETF Prices

In [ ]:
prices = get_etf_data(TICKERS, START_DATE, END_DATE)

In [462]:
prices.head()

Ticker,BIL,GLD,SPY,TIP,TLT
Date,,,,,
2015-01-02,76.333511,114.080002,171.568024,85.979111,96.881851
2015-01-05,76.350204,115.800003,168.469604,86.047729,98.403732
2015-01-06,76.333511,117.120003,166.882736,86.055382,100.176682
2015-01-07,76.333511,116.430000,168.962357,86.124031,99.978813
2015-01-08,76.333511,115.940002,171.960571,85.986748,98.654800


### 🧮 Volatility and Weight Calculation

Functions for calculating rolling volatility and daily risk-parity weights

In [463]:
def calculate_rolling_volatility(tickers: list[str], start: str, end: str, window: int):
    """
    Calculates rolling 126-day volatility for given tickers and date range.
    
    Parameters:
        tickers (list[str]): U.S. equity ticker symbols
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format
        window (int): Length of lookback period in number of days

    Returns:
        pd.DataFrame: DataFrame with date-indexed rolling 126-day volatility
    """

    # Fetch price data
    data = get_etf_data(tickers, start, end)

    # Compute daily return
    returns = data.pct_change()

    # Compute rolling volatility
    rolling_vol = returns.rolling(window=window).std()

    return rolling_vol

In [464]:
def calculate_daily_risk_parity_weights(tickers: list[str] = TICKERS, start: str = START_DATE, end: str = END_DATE, window: int = VOL_LOOKBACK):
    """
    Calculates daily risk-parity weights for given tickers and date range.
    
    Parameters:
        tickers (list[str]): U.S. equity ticker symbols
        start (str): Start date in 'YYYY-MM-DD' format
        end (str): End date in 'YYYY-MM-DD' format
        window (int): Length of lookback period in number of days

    Returns:
        pd.DataFrame: DataFrame with date-indexed risk-parity weights
    """

    # Fetch rolling volatility
    vol = calculate_rolling_volatility(tickers, start, end, window)

    # Invert volatility
    inv_vol = 1 / vol.replace(0, np.nan)  # Replace zero vol with NaN to avoid inf

    # Normalize rows so weights sum to 1
    weights = inv_vol.div(inv_vol.sum(axis=1), axis=0)  # Fill NaN weights with zero

    return weights

Portfolio Weights

In [ ]:
portfolio_weights = calculate_daily_risk_parity_weights()

In [466]:
portfolio_weights.dropna().head()

Ticker,BIL,GLD,SPY,TIP,TLT
Date,,,,,
2015-07-06,0.907874,0.017555,0.020895,0.039061,0.014616
2015-07-07,0.907447,0.017626,0.021319,0.038926,0.014683
2015-07-08,0.908006,0.017643,0.020909,0.038705,0.014736
2015-07-09,0.908296,0.017669,0.021135,0.038341,0.014558
2015-07-10,0.908260,0.017680,0.021372,0.038165,0.014522


### 📜 Backtesting

Functions for calculating risk-parity portfolio returns and performance metrics

In [467]:
def calculate_portfolio_returns(weights: pd.DataFrame, closing_prices: pd.DataFrame):
    """
    Calculates daily risk-parity portfolio returns for given weights and total-return ETF prices
    
    Parameters:
        weights (pd.DataFrame): Daily risk-parity weights of the portfolio
        closing_prices (pd.DataFrame): Daily adjusted closing price of each ETF

    Returns:
        pd.DataFrame: DataFrame with date-indexed daily returns
    """

    # Compute daily return
    returns = closing_prices.pct_change().dropna()

    # Shift weights to avoid look-ahead bias
    weights_lagged = weights.shift(1).dropna()

    # Align returns with weights
    returns = returns.loc[weights_lagged.index]

    # Calculate portfolio returns
    portfolio_returns = (weights_lagged * returns).sum(axis=1).to_frame()

    # Rename the column to 'returns'
    portfolio_returns.columns = ['returns']

    # Calculate cumulative returns
    portfolio_returns['cumulative_returns'] = (1 + portfolio_returns['returns']).cumprod()

    return portfolio_returns

In [468]:
def calculate_portfolio_metrics(portfolio_returns: pd.DataFrame, risk_free_rate: float = 0.0) -> dict:
    """
    Computes annualized return, volatility, Sharpe ratio, and max drawdown, assuming 252 trading days/year

    Parameters:
        portfolio_returns (pd.DataFrame): Daily returns of portfolio
        risk_free_rate (float): Daily risk-free rate in decimal (e.g., 0.0 or 0.0001 for 2.5% annualized)

    Returns:
        dict: Dictionary of metrics
    """

    # Calculates number of years of data
    num_years = (portfolio_returns.index[-1] - portfolio_returns.index[0]).days / 365.25

    annualized_return = float(portfolio_returns['cumulative_returns'].iloc[-1] ** (1 / num_years) - 1)

    annualized_volatility = float(portfolio_returns['returns'].std() * np.sqrt(252))

    sharpe_ratio = float(((portfolio_returns['returns'] - risk_free_rate).mean() / portfolio_returns['returns'].std()) * np.sqrt(252))

    max_drawdown = float((portfolio_returns['cumulative_returns'] / portfolio_returns['cumulative_returns'].cummax() - 1).min())

    return {
        'Annualized Return': annualized_return,
        'Annualized Volatility': annualized_volatility,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown': max_drawdown
    }


Function for backtesting

In [469]:
def backtest_portfolio(weights: pd.DataFrame, closing_prices: pd.DataFrame):
    """
    Backtest risk-parity portfolio by calculating portfolio metrics
    
    Parameters:
        weights (pd.DataFrame): Daily risk-parity weights of the portfolio
        closing_prices (pd.DataFrame): Daily adjusted closing price of each ETF

    Returns:
        Tuple[pd.DataFrame, dict]: Daily portfolio returns as DataFrame and summary performance metrics as a dictionary
    """

    returns = calculate_portfolio_returns(weights, closing_prices)

    metrics = calculate_portfolio_metrics(returns)

    return returns, metrics

Returns and Performance Metrics

In [470]:
portfolio_returns, portfolio_metrics = backtest_portfolio(portfolio_weights, prices)

In [471]:
portfolio_returns.head()

,returns,cumulative_returns
Date,,
2015-07-07,-0.000006,0.999994
2015-07-08,-0.000130,0.999864
2015-07-09,-0.000459,0.999405
2015-07-10,-0.000125,0.999280
2015-07-13,-0.000022,0.999258


In [472]:
portfolio_metrics

{'Annualized Return': 0.021750571038681832,
 'Annualized Volatility': 0.0061590177344481665,
 'Sharpe Ratio': 3.502098909340861,
 'Max Drawdown': -0.007587787996374162}

### 📈 Performance Plots

Functions for plotting equity curve and weights evolution

In [473]:
def plot_equity_curve(returns: pd.DataFrame):
    """
    Plot equity curve for given portfolio returns
    
    Parameters:
        portfolio_returns (pd.DataFrame): Daily returns of portfolio
    """
    
    cumulative_returns = (1 + returns['returns']).cumprod()

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=cumulative_returns.index,
        y=cumulative_returns,
        mode='lines',
        name='Equity Curve',
        line=dict(color='green')
    ))

    fig.update_layout(
        title='Portfolio Equity Curve',
        xaxis_title='Date',
        yaxis_title='Cumulative Return (per $1)',
        template='plotly_white'
    )
    fig.show()

In [474]:
def plot_weights(weights: pd.DataFrame):
    """
    Plot weight evolution of each ETF over time
    
    Parameters:
        weights (pd.DataFrame): Daily risk-parity weights of the portfolio
    """
    
    fig = go.Figure()

    for col in weights.columns:
        fig.add_trace(go.Scatter(
            x=weights.index,
            y=weights[col],
            mode='lines',
            name=col
        ))

    fig.update_layout(
        title='Portfolio Weights over Time',
        xaxis_title='Date',
        yaxis_title='Weight',
        yaxis=dict(range=[0, 1]),
        template='plotly_white'
    )
    fig.show()


Plots

In [475]:
plot_equity_curve(portfolio_returns)

In [476]:
plot_weights(portfolio_weights)

### 🧪 Stress-Testing Under Extreme Scenarios

Stress Scenarios Parameters

In [477]:
STRESS_EVENTS: dict[str, tuple[str]] = {
    "March 2020 COVID-19 Crash (Feb-Apr 2020)": ("2020-02-19", "2020-03-23"),
    "2022 Russian-Ukraine War sell-off": ("2022-02-16", "2022-03-15"),
    "2025 Feb-April sell-off": ("2025-02-01", "2025-04-08")
}

EXTREME_SHOCK = {'SPY': -0.3, 'TLT': 0.1, 'GLD': -0.15}  # –30% equity, +10% rates move, –15% commodity move

Function for stress-testing the portfolio by recalculating returns and performance metrics

In [478]:
def stress_test_period(weights: pd.DataFrame, closing_prices: pd.DataFrame, start_date: str, end_date: str):
    """
    Backtest risk-parity portfolio by calculating portfolio metrics
    
    Parameters:
        weights (pd.DataFrame): Daily risk-parity weights of the portfolio
        closing_prices (pd.DataFrame): Daily adjusted closing price of each ETF
        start_date (str): Start of stress period (YYYY-MM-DD).
        end_date (str): End of stress period (YYYY-MM-DD).

    Returns:
        Tuple[pd.DataFrame, dict[str, float|int]]: Daily portfolio returns under stress as DataFrame and summary performance metrics as a dictionary
    """

    # Calculate portfolio's entire daily returns
    full_returns = calculate_portfolio_returns(weights, closing_prices)

    # Get portfolio's returns and cumulative returns during stress window
    stress_returns = full_returns.loc[start_date:end_date].copy()

    # Calculate portfolio's performance metrics during stress window
    stress_metrics = calculate_portfolio_metrics(stress_returns)
    
    # Find pre-stress peak
    pre_stress_peak = full_returns['cumulative_returns'].loc[:start_date].max()

    # Calculate portfolio's post_stress returns
    post_stress_returns = full_returns['cumulative_returns'].loc[end_date:]

    # Calculate recovery date and time
    recovery_date = post_stress_returns[post_stress_returns >= pre_stress_peak].first_valid_index()
    recovery_time = post_stress_returns.index.get_loc(recovery_date) if recovery_date else None

    # Add metric into dictionary
    stress_metrics['Recovery Time (Trading days)'] = recovery_time

    return stress_returns, stress_metrics

Function for simulating shock

In [479]:
def simulate_shock(weights: pd.DataFrame, shock: dict[str, float], date: str | None = None):
    """
    Simulate extreme shock to estimate portfolio loss for given weights and day
    
    Parameters:
        weights (pd.DataFrame): Daily risk-parity weights of the portfolio
        shock (dict[str, float]): The shock to each asset
        date (str | None): Date of to simulate the shock for (default to first day)

    Returns:
        float: Portoflio loss from the shock
    """

    return float((weights.dropna().loc[date] * pd.Series(shock)).sum()) if date else float((weights.dropna().iloc[0] * pd.Series(shock)).sum())

Functions for calculating full-history and rolling VaR/CVaR

In [480]:
def calculate_full_hist_var_cvar(returns: pd.DataFrame, confidence: float = 0.95):
    """
    Calculate full-history Value-at-Risk and CVaR for given portfolio returns and confidence level
    
    Parameters:
        returns (pd.DataFrame): Daily returns of portfolio
        confidence (float): Confidence level of calculation (default to 0.95)

    Returns:
        dict[str, float]: Full-history VaR and CVaR
    """

    var = np.percentile(returns['returns'], 100 * (1 - confidence))
    cvar = returns['returns'][returns['returns'] <= var].mean()
    
    return {'VaR': float(var), 'CVaR': float(cvar)}

In [481]:
def calculate_rolling_var_cvar(returns: pd.DataFrame, period: int = 252, confidence: float = 0.95):
    """
    Calculate rolling Value-at-Risk and CVaR for given portfolio returns and confidence level
    
    Parameters:
        returns (pd.DataFrame): Daily returns of portfolio
        period (int): Length of lookback period in number of days (default to 252)
        confidence (float): Confidence level of calculation (default to 0.95)

    Returns:
        pd.DataFrame: Date-Indexed dataframe of rolling VaR and CVaR
    """

    rolling_var = []
    rolling_cvar = []

    for i in range(period, len(returns)):
        window_returns = returns[i - period:i]

        var_and_cvar = calculate_full_hist_var_cvar(window_returns)

        rolling_var.append(var_and_cvar['VaR'])
        rolling_cvar.append(var_and_cvar['CVaR'])
    
    dates = returns.index[period:]
    rolling_var_series = pd.Series(rolling_var, index = dates, name = 'VaR')
    rolling_cvar_series = pd.Series(rolling_cvar, index = dates, name = 'CVaR')

    return pd.concat([rolling_var_series, rolling_cvar_series], axis=1)

Stress Returns and Performance Metrics

In [499]:
stresses_performances = {}
for event in STRESS_EVENTS:
    stress_returns, stress_metrics = stress_test_period(portfolio_weights, prices, STRESS_EVENTS[event][0], STRESS_EVENTS[event][1])
    stresses_performances[event] = (stress_returns, stress_metrics)

shock_loss = simulate_shock(portfolio_weights, EXTREME_SHOCK)

var_and_cvar = calculate_full_hist_var_cvar(portfolio_returns)

rolling_var_and_cvar = calculate_rolling_var_cvar(portfolio_returns)

In [483]:
stresses_performances["March 2020 COVID-19 Crash (Feb-Apr 2020)"][0].head()

,returns,cumulative_returns
Date,,
2020-02-19,0.000140,1.075143
2020-02-20,0.000163,1.075318
2020-02-21,0.000444,1.075795
2020-02-24,-0.000123,1.075663
2020-02-25,-0.000665,1.074948


In [484]:
stresses_performances["March 2020 COVID-19 Crash (Feb-Apr 2020)"][1]

{'Annualized Return': 1.2107351791688816,
 'Annualized Volatility': 0.013035538710500108,
 'Sharpe Ratio': -0.5074318894477751,
 'Max Drawdown': -0.0066734838790748485,
 'Recovery Time (Trading days)': 1}

In [485]:
stresses_performances["2022 Russian-Ukraine War sell-off"][0].head()

,returns,cumulative_returns
Date,,
2022-02-16,0.000261,1.080356
2022-02-17,-0.000007,1.080348
2022-02-18,0.000230,1.080596
2022-02-22,0.000032,1.080630
2022-02-23,-0.000341,1.080262


In [486]:
stresses_performances["2022 Russian-Ukraine War sell-off"][1]

{'Annualized Return': 1.8606450536263872,
 'Annualized Volatility': 0.006068740392197597,
 'Sharpe Ratio': 1.4599516778419739,
 'Max Drawdown': -0.001729816370763082,
 'Recovery Time (Trading days)': 167}

In [487]:
stresses_performances["2025 Feb-April sell-off"][0].head()

,returns,cumulative_returns
Date,,
2025-02-03,0.000484,1.217061
2025-02-04,0.000391,1.217537
2025-02-05,0.000764,1.218467
2025-02-06,0.000116,1.218608
2025-02-07,-0.000181,1.218387


In [488]:
stresses_performances["2025 Feb-April sell-off"][1]

{'Annualized Return': 2.1747081108720323,
 'Annualized Volatility': 0.00671090639055371,
 'Sharpe Ratio': 5.2808952702926755,
 'Max Drawdown': -0.0024344071980284543,
 'Recovery Time (Trading days)': 0}

In [500]:
print("Shock Loss:", shock_loss)

Shock Loss: -0.007439968849115339


In [501]:
var_and_cvar

{'VaR': -0.0005418837784754062, 'CVaR': -0.0007485657986657105}

In [502]:
rolling_var_and_cvar.head()

,VaR,CVaR
Date,,
2016-07-06,-0.000549,-0.000717
2016-07-07,-0.000549,-0.000717
2016-07-08,-0.000549,-0.000717
2016-07-11,-0.000549,-0.000717
2016-07-12,-0.000549,-0.000717


### 📉 Stress Performance Plots

Function for plotting equity curve overlays alongisde full-history curve for all stress windows

In [492]:
def plot_equity_curve_with_stress(portfolio_returns: pd.DataFrame, stress_windows: dict[str, tuple[str]]):
    """
    Plot full equity curve with overlays for stress scenarios.

    Parameters:
        portfolio_returns (pd.DataFrame): Portfolio's entire daily returns and equity 
        stress_windows (dict[str, tuple[str]]): Dict of {label: (start_date, end_date)}
    """

    # Create plot
    fig = go.Figure()

    # Full equity curve
    fig.add_trace(go.Scatter(
        x=portfolio_returns['cumulative_returns'].index, y=portfolio_returns['cumulative_returns'].values,
        mode='lines', name='Full Equity Curve',
        line=dict(color='yellow', width=2)
    ))

    # Add stress overlays
    for label, (start, end) in stress_windows.items():
        stress_curve = portfolio_returns['cumulative_returns'].loc[start:end]

        fig.add_trace(go.Scatter(
            x=stress_curve.index,
            y=stress_curve.values,
            mode='lines',
            name=label,
            line=dict(dash='dashdot'),
        ))

    # Layout
    fig.update_layout(
        title="Equity Curve with Stress Period Overlays",
        xaxis_title="Date",
        yaxis_title="Portfolio Value",
        legend_title="Legend",
        template="plotly_white"
    )

    fig.show()

Function for plotting rolling VaR and CVaR

In [493]:
def plot_rolling_var_and_cvar(rolling_var_and_cvar_series: pd.DataFrame):
    """
    Plot rolling VaR and CVaR

    Parameters:
        rolling_var_and_cvar_series (pd.DataFrame): Date-indexed rolling VaR and CVaR
    """
    
    # Create plot
    fig = go.Figure()

    # Add curves
    fig.add_trace(go.Scatter(x=rolling_var_and_cvar_series.index, y=rolling_var_and_cvar_series['VaR'], name='Rolling VaR (95%)'))
    fig.add_trace(go.Scatter(x=rolling_var_and_cvar_series.index, y=rolling_var_and_cvar_series['CVaR'], name='Rolling CVaR (95%)'))

    # Layout
    fig.update_layout(title="Rolling 252-Day VaR and CVaR", xaxis_title="Date", yaxis_title="Return", template="plotly_white")
    fig.show()

Plots

In [494]:
plot_equity_curve_with_stress(portfolio_returns, STRESS_EVENTS)

In [495]:
plot_rolling_var_and_cvar(rolling_var_and_cvar)